# Applied Data Science Capstone - week 3

## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files

In [2]:
import bs4 as bs # import Beatifule Soup library
import urllib.request
import lxml.html # new import for local jupyter
import html5lib # new import for local jupyter

In [3]:
# read table with Pandas
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)

# format table
df_canada = dfs[0] # extract postcode table only
df_canada.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

## Assignment 1

### develop a table with (Postal code) - (Borough) - (Neighbourhood)

In [4]:
# eliminate "Not assigned" at Borough, sort with Postcode, reset_index
df_canada2 = df_canada.query('Borough != "Not assigned"')
df_canada3 = df_canada2.sort_values('Postcode', ascending=True) 
df_canada3.reset_index(inplace=True, drop=True)

# replace "Not assigned" at Neighbourhood with Borough
N = len(df_canada3)
for i in range(N):
    if df_canada3.iloc[i,2] == "Not assigned":
        df_canada3.iloc[i,2] = df_canada3.iloc[i,1]
# df_canada3

# concatinate Neighbourhood for same Postcode 
N = len(df_canada3)
i = 1 # compare from the 2nd line
while(i < N):
    if df_canada3.iloc[i,0] == df_canada3.iloc[i-1,0]:
        df_canada3.iloc[i,2] = df_canada3.iloc[i,2] + ',' + df_canada3.iloc[i-1,2]
        df_canada3.iloc[i-1,1] = "Not assigned" # place "Not assigned" for later elimination
    i = i + 1
#df_canada3
    
# eliminate "Not assigned" at Borough again, reset_index
df_canada4 = df_canada3.query('Borough != "Not assigned"')
df_canada4.reset_index(inplace=True, drop=True)

df_canada4

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"West Hill,Morningside,Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park,Ionview,East Birchmount Park"
7,M1L,Scarborough,"Clairlea,Oakridge,Golden Mile"
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [5]:
df_canada4.shape

(103, 3)

## Assignment 2

### develop a table with (Postal code) - (Borough) - (Neighbourhood) - (Latitudo) -(Longitude)

In [6]:
# read CSV
path='https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_postcode = pd.read_csv(path)

In [7]:
# df_postcode.head()
df_postcode.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [8]:
# merge dataframes
df_canada5 = pd.merge(df_canada4, df_postcode, left_on='Postcode', right_on='Postal Code')
df_canada6 = df_canada5.drop('Postal Code', axis=1)
df_canada6

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"West Hill,Morningside,Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park,Ionview,East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Oakridge,Golden Mile",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


## Assignment 3

### select postcode area which contains a word "Toronto" at Borough

In [9]:
df_canada7 = df_canada6.query('Borough.str.contains("Toronto")', engine='python')
df_canada7.reset_index(inplace=True, drop=True)
df_canada7

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar,The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Summerhill East,Moore Park",43.689574,-79.383160
9,M4V,Central Toronto,"Forest Hill SE,Deer Park,Summerhill West,Rathn...",43.686412,-79.400049


In [10]:
# find venues of the area (example)
# Define Foursquare Credentials and Version
CLIENT_ID = 'YEPDOJTVVOTCUGYYITPOV0GKBJBD2YCQKKHQTJ2EU2X14OAD' # your Foursquare ID
CLIENT_SECRET = 'K2GE1JRL0I45JPDQFYHCKYUYH1ONP3LHDUZLGGICJW2AMWIC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YEPDOJTVVOTCUGYYITPOV0GKBJBD2YCQKKHQTJ2EU2X14OAD
CLIENT_SECRET:K2GE1JRL0I45JPDQFYHCKYUYH1ONP3LHDUZLGGICJW2AMWIC


In [11]:
### Get the neighborhood (Postcode)'s latitude and longitude values.
#df_canada7.head()
df_canada7.loc[0, ['Postcode', 'Borough' ,'Neighbourhood']]

Postcode                  M4E
Borough          East Toronto
Neighbourhood     The Beaches
Name: 0, dtype: object

In [12]:
neighborhood_latitude = df_canada7.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_canada7.loc[0, 'Longitude'] # neighborhood longitude value

#neighborhood_name = manhattan_data.loc[0, 'Neighbourhood'] # neighborhood name
neighborhood_post = df_canada7.loc[0, 'Postcode'] # use Neighborhood instead of Postcode number
neighborhood_name = df_canada7.loc[0, 'Neighbourhood'] # use Neighborhood instead of Postcode number

print('Latitude and longitude values of {}, {} are {}, {}.'.format(neighborhood_post, neighborhood_name,
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of M4E, The Beaches are 43.67635739999999, -79.2930312.


### define Foursquare API parameters and examine the resutls of Foursquare API access

In [13]:
# type your answer here
radius = 200
LIMIT = 20
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=YEPDOJTVVOTCUGYYITPOV0GKBJBD2YCQKKHQTJ2EU2X14OAD&client_secret=K2GE1JRL0I45JPDQFYHCKYUYH1ONP3LHDUZLGGICJW2AMWIC&v=20180605&ll=43.67635739999999,-79.2930312&radius=200&limit=20'

In [14]:
# Send the GET request and examine the resutls of Foursquare API
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ce1137e6a6071493b714fa5'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.67815740179999,
    'lng': -79.2905470857962},
   'sw': {'lat': 43.674557398199994, 'lng': -79.2955153142038}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4dbc8fe96a23e294ba3237bd',
       'name': 'Glen Stewart Park',
       'location': {'address': '131 Glen Manor Drive',
        'lat': 43.67527822698259,
        'lng': -79.2946465494184,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67527822698259,
          'lng': -79.2946465494184}],
        'distance': 177,
        'cc': 'CA',
 

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
venues = results['response']['groups'][0]['items']
    
# nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues = pd.io.json.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Stewart Park,Park,43.675278,-79.294647
1,Glen Stewart Ravine,Other Great Outdoors,43.676300,-79.294784
2,Balmy Beach Playground,Playground,43.676078,-79.290805
3,Glen Manor Ravine,Trail,43.676821,-79.293942


In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


### Explore Neighborhoods (postcode area) in Totonto (postcode area which contains a word "Toronto")

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([( 
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Create a new dataframe called "toronto_venues" which has venue information

In [19]:
# Now write the code to run the above function on each neighborhood 
# and create a new dataframe called toronto_venues

toronto_venues = getNearbyVenues(names=df_canada7['Postcode'], 
                                   latitudes=df_canada7['Latitude'],
                                   longitudes=df_canada7['Longitude']
                                  )


In [20]:
# Check the size of the resulting dataframe
print(toronto_venues.shape)
toronto_venues.head()

(620, 7)


,Postcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,M4E,43.676357,-79.293031,Guru Raghavendra Ji,43.680187,-79.292337,Astrologer
3,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678730,-79.297478,Grocery Store
4,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


### Check total number of venues for each Neighbourhood (postalcode area)

In [21]:
# Let's check how many venues were returned for each neighborhood
toronto_venues.groupby('Postcode').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode,,,,,,
M4E,5,5,5,5,5,5
M4K,20,20,20,20,20,20
M4L,20,20,20,20,20,20
M4M,20,20,20,20,20,20
M4N,4,4,4,4,4,4
M4P,7,7,7,7,7,7
M4R,18,18,18,18,18,18
M4S,20,20,20,20,20,20
M4T,3,3,3,3,3,3


In [22]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 168 uniques categories.


### Analyze each Neighborhood (Postalcode area)

In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postcode'] = toronto_venues['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postcode,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Market,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pool,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [24]:
# And let's examine the new dataframe size.
toronto_onehot.shape

(620, 169)

In [25]:
# let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped

,Postcode,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Market,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pool,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,M4E,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.2,0.0000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.200000,0.000000,0.000000,0.000000,0.2,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.20,0.00,0.0000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.200000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000
1,M4K,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.0000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.050000,0.000000,0.000000,0.050000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.050000,0.00,0.000000,0.00,0.00,0.050000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,

In [26]:
# Let's confirm the new size
toronto_grouped.shape

(38, 169)

### Show postalcode area and with the top 5 most common venues

In [27]:
# Print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Postcode']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Postcode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
               venue  freq
0      Grocery Store   0.2
1         Astrologer   0.2
2                Pub   0.2
3  Health Food Store   0.2
4       Neighborhood   0.2


----M4K----
                     venue  freq
0         Greek Restaurant  0.25
1           Ice Cream Shop  0.10
2       Italian Restaurant  0.10
3              Yoga Studio  0.05
4  Fruit & Vegetable Store  0.05


----M4L----
              venue  freq
0    Sandwich Place  0.10
1               Pub  0.05
2  Sushi Restaurant  0.05
3       Pizza Place  0.05
4       Coffee Shop  0.05


----M4M----
                venue  freq
0         Coffee Shop  0.15
1              Bakery  0.10
2                Café  0.10
3         Cheese Shop  0.05
4  Chinese Restaurant  0.05


----M4N----
         venue  freq
0         Lake  0.25
1  Swim School  0.25
2         Park  0.25
3     Bus Line  0.25
4      Airport  0.00


----M4P----
               venue  freq
0     Clothing Store  0.14
1     Breakfast Spot  0.14
2              Hotel  0.14


### Place venue information to a pandas dataframe for clustering

In [28]:
# sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
# create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = toronto_grouped['Postcode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Grocery Store,Pub,Health Food Store,Neighborhood,Astrologer,Yoga Studio,Cosmetics Shop,Discount Store,Diner,Dessert Shop
1,M4K,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Restaurant,Bookstore,Brewery,Cosmetics Shop,Dessert Shop,Fruit & Vegetable Store,Juice Bar
2,M4L,Sandwich Place,Burrito Place,Italian Restaurant,Sushi Restaurant,Ice Cream Shop,Burger Joint,Liquor Store,Fish & Chips Shop,Pub,Brewery
3,M4M,Coffee Shop,Café,Bakery,Ice Cream Shop,Comfort Food Restaurant,Cheese Shop,Neighborhood,Middle Eastern Restaurant,Sandwich Place,Seafood Restaurant
4,M4N,Park,Swim School,Bus Line,Lake,Yoga Studio,Cuban Restaurant,Dog Run,Discount Store,Diner,Dessert Shop


### Cluster Neighborhoods (with postalcode area)

In [30]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [31]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 4, 0, 2, 0, 4, 4, 3, 0], dtype=int32)

In [32]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
neighborhoods_venues_sorted[0:5]

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Grocery Store,Pub,Health Food Store,Neighborhood,Astrologer,Yoga Studio,Cosmetics Shop,Discount Store,Diner,Dessert Shop
1,M4K,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Restaurant,Bookstore,Brewery,Cosmetics Shop,Dessert Shop,Fruit & Vegetable Store,Juice Bar
2,M4L,Sandwich Place,Burrito Place,Italian Restaurant,Sushi Restaurant,Ice Cream Shop,Burger Joint,Liquor Store,Fish & Chips Shop,Pub,Brewery
3,M4M,Coffee Shop,Café,Bakery,Ice Cream Shop,Comfort Food Restaurant,Cheese Shop,Neighborhood,Middle Eastern Restaurant,Sandwich Place,Seafood Restaurant
4,M4N,Park,Swim School,Bus Line,Lake,Yoga Studio,Cuban Restaurant,Dog Run,Discount Store,Diner,Dessert Shop


In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_canada7 # df_canada7

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Postcode'), on='Postcode')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Grocery Store,Pub,Health Food Store,Neighborhood,Astrologer,Yoga Studio,Cosmetics Shop,Discount Store,Diner,Dessert Shop
1,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188,4,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Restaurant,Bookstore,Brewery,Cosmetics Shop,Dessert Shop,Fruit & Vegetable Store,Juice Bar
2,M4L,East Toronto,"India Bazaar,The Beaches West",43.668999,-79.315572,4,Sandwich Place,Burrito Place,Italian Restaurant,Sushi Restaurant,Ice Cream Shop,Burger Joint,Liquor Store,Fish & Chips Shop,Pub,Brewery
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Café,Bakery,Ice Cream Shop,Comfort Food Restaurant,Cheese Shop,Neighborhood,Middle Eastern Restaurant,Sandwich Place,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Swim School,Bus Line,Lake,Yoga Studio,Cuban Restaurant,Dog Run,Discount Store,Diner,Dessert Shop


### Create a map to show clusters of Nighbouthood (postalcode area)

In [34]:
# Use geopy library to get the latitude and longitude values of Toronto City
# address = 'Toronto City'

# geolocator = Nominatim(user_agent="ny_explorer")
# location = geolocator.geocode(address)
latitude = 43.6532
longitude =  -79.3831
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6532, -79.3831.


In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine features of each Cluster

#### Cluster 1

In [36]:
# cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Grocery Store,Pub,Health Food Store,Neighborhood,Astrologer,Yoga Studio,Cosmetics Shop,Discount Store,Diner,Dessert Shop
3,East Toronto,0,Coffee Shop,Café,Bakery,Ice Cream Shop,Comfort Food Restaurant,Cheese Shop,Neighborhood,Middle Eastern Restaurant,Sandwich Place,Seafood Restaurant
5,Central Toronto,0,Clothing Store,Park,Breakfast Spot,Sandwich Place,Food & Drink Shop,Hotel,Gym,Coffee Shop,Diner,Dessert Shop
9,Central Toronto,0,Coffee Shop,Pub,Pizza Place,Vietnamese Restaurant,Fried Chicken Joint,American Restaurant,Sports Bar,Bagel Shop,Light Rail Station,Liquor Store
11,Downtown Toronto,0,Café,Gastropub,Indian Restaurant,Pet Store,Caribbean Restaurant,Pub,Butcher,Restaurant,Market,Deli / Bodega
13,Downtown Toronto,0,Coffee Shop,Park,Breakfast Spot,Bakery,Historic Site,Dessert Shop,Performing Arts Venue,Farmers Market,Restaurant,Pub
14,Downtown Toronto,0,Café,Pizza Place,Taco Place,Clothing Store,Music Venue,Movie Theater,Coffee Shop,Plaza,Burrito Place,Burger Joint
15,Downtown Toronto,0,Gastropub,Coffee Shop,Restaurant,Japanese Restaurant,Hotel,Cosmetics Shop,Café,Creperie,Middle Eastern Restaurant,Italian Restaurant
16,Downtown Toronto,0,Farmers Market,Seafood Restaurant,Park,Tea Room,Beer Bar,Basketball Stadium,Breakfast Spot,Steakhouse,Liquor Store,Fish Market
18,Downtown Toronto,0,Steakhouse,Asian Restaurant,Greek Restaurant,Gym / Fitness Center,Food Court,Hotel,Concert Hall,Coffee Shop,Neighborhood,Opera House


#### Cluster 2

In [37]:
# cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,Garden,Yoga Studio,Cosmetics Shop,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega,Dance Studio,Cuban Restaurant


#### Cluster 3

In [38]:
# cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Park,Swim School,Bus Line,Lake,Yoga Studio,Cuban Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
23,Central Toronto,2,Park,Jewelry Store,Trail,Sushi Restaurant,Bus Line,Yoga Studio,Creperie,Discount Store,Diner,Dessert Shop


#### Cluster 4

In [39]:
# cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,3,Playground,Park,Tennis Court,Clothing Store,Cosmetics Shop,Discount Store,Diner,Dessert Shop,Deli / Bodega,Dance Studio
10,Downtown Toronto,3,Park,Playground,Trail,Convenience Store,Discount Store,Diner,Dessert Shop,Deli / Bodega,Dance Studio,Cuban Restaurant


#### Cluster 5

In [40]:
# cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,4,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Restaurant,Bookstore,Brewery,Cosmetics Shop,Dessert Shop,Fruit & Vegetable Store,Juice Bar
2,East Toronto,4,Sandwich Place,Burrito Place,Italian Restaurant,Sushi Restaurant,Ice Cream Shop,Burger Joint,Liquor Store,Fish & Chips Shop,Pub,Brewery
6,Central Toronto,4,Coffee Shop,Clothing Store,Yoga Studio,Sporting Goods Shop,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Cosmetics Shop,Park
7,Central Toronto,4,Dessert Shop,Sushi Restaurant,Italian Restaurant,Greek Restaurant,Gym,Indian Restaurant,Diner,Coffee Shop,Park,Café
12,Downtown Toronto,4,Diner,Mexican Restaurant,Breakfast Spot,Juice Bar,Salon / Barbershop,Bubble Tea Shop,Japanese Restaurant,Restaurant,Ethiopian Restaurant,Ramen Restaurant
17,Downtown Toronto,4,Coffee Shop,Italian Restaurant,Pizza Place,Sushi Restaurant,Park,Chinese Restaurant,Gastropub,Modern European Restaurant,Ramen Restaurant,Bubble Tea Shop
25,Downtown Toronto,4,Bookstore,Bakery,Japanese Restaurant,Restaurant,Café,Beer Bar,Sandwich Place,Dessert Shop,Italian Restaurant,Sushi Restaurant
32,West Toronto,4,Bar,Pizza Place,Yoga Studio,Record Shop,Art Gallery,Asian Restaurant,Brewery,Cocktail Bar,Cuban Restaurant,French Restaurant
34,West Toronto,4,Café,Arts & Crafts Store,Mexican Restaurant,Speakeasy,Park,Bar,Fast Food Restaurant,Italian Restaurant,Flea Market,Diner
35,West Toronto,4,Gift Shop,Breakfast Spot,Restaurant,Bar,Bank,Bookstore,Dessert Shop,Italian Restaurant,Eastern European Restaurant,Cuban Restaurant


## Summary of Toronto city clusters

#### Cluster 1 has "business" and "commercial" area neaby and has variety of facilities and shops
#### Cluster 2 is a group which has gardens 
#### Cluster 3 is a group which has parks, 
#### Cluster 4 is a group which has playground and park
#### Cluster 5 is a group which has a lot of food shops